In [1]:
sc

In [151]:
import numpy as np
import pandas as pd
import os
import pyspark.sql.functions as sf
from pyspark.sql import Row
from nltk.corpus import stopwords
from textstat.textstat import textstat
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

ImportError: No module named udf

---
# Yelp Datasets

In [10]:
path = '/Users/nnhoucheng/Dropbox/Yelp Project/Data/'
os.listdir(path)

['.ipynb_checkpoints',
 'Pittsburgh',
 'Yelp Shapefile',
 'Yelp.xls',
 'yelp_academic_dataset_business.json',
 'yelp_academic_dataset_checkin.json',
 'yelp_academic_dataset_review.json',
 'yelp_academic_dataset_tip.json',
 'yelp_academic_dataset_user.json']

## User profile

In [11]:
user = sqlContext.read.load(path + 'yelp_academic_dataset_user.json', format='json')

In [15]:
user.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliments: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- cute: long (nullable = true)
 |    |-- funny: long (nullable = true)
 |    |-- hot: long (nullable = true)
 |    |-- list: long (nullable = true)
 |    |-- more: long (nullable = true)
 |    |-- note: long (nullable = true)
 |    |-- photos: long (nullable = true)
 |    |-- plain: long (nullable = true)
 |    |-- profile: long (nullable = true)
 |    |-- writer: long (nullable = true)
 |-- elite: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- fans: long (nullable = true)
 |-- friends: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- votes: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- funny: long (nullable = true)

### Filter review_count < 50

In [30]:
activeuser = user.filter(user.review_count >= 50).select('user_id').collect()

In [33]:
activeuser = map(lambda x: x['user_id'], activeuser)

In [34]:
len(activeuser)

72273

## Review text

In [24]:
review =sqlContext.read.load(path + 'yelp_academic_dataset_review.json', format='json')

In [26]:
review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- votes: struct (nullable = true)
 |    |-- cool: long (nullable = true)
 |    |-- funny: long (nullable = true)
 |    |-- useful: long (nullable = true)



### Merge

In [41]:
url = 'https://raw.githubusercontent.com/lx565/YelpFakeReviews/master/yelp.csv'
df_review = pd.read_csv(url, header=None)
df_review.columns = ['user_id', 'cool', 'funny', 'useful', 'stars']
df_review.head()

,user_id,cool,funny,useful,stars
0,JPPhyFE-UE453zA6K0TVgw,0,0,0,4
1,fhNxoMwwTipzjO8A9LFe8Q,0,0,1,3
2,tAKjY3bQXH51msJbOHYPmQ,2,0,3,5
3,Sjb5e5-gKoLXueFDMc2R8Q,0,0,0,4
4,2rSeth60_CuWN3ZJ4k41lg,19,11,22,3


In [81]:
df_review_u = df_review.groupby('user_id').mean().reset_index()
df_review_u = df_review_u.merge(df_review.groupby('user_id').count()['cool']\
                                .reset_index().rename(columns={'cool':'review_count'}),
                                on='user_id')

In [82]:
df_review_u.head()

,user_id,cool,funny,useful,stars,review_count
0,--65q1FpAL_UQtVZ2PTGew,2.695364,1.907285,3.807947,4.046358,151
1,-0itF0VWVBe3k2AdfUReGA,1.734043,1.500000,2.393617,3.670213,94
2,-2PFmYGWB6rQhZtRh6PlPA,0.183333,0.166667,0.675000,3.283333,120
3,-3akdU5UTDn6dwiTCO2cAw,0.736842,0.631579,1.447368,3.934211,76
4,-4AMRN-7RtozD9s8ZsH04g,1.644068,2.745763,2.644068,3.593220,59


In [49]:
df_users = list(df_review.user_id.unique())

In [74]:
user_p = user.filter(user.user_id.isin(df_users))

In [85]:
user_p = user_p.select('user_id',
                       'name',
                       'yelping_since',
                       'fans',
                       sf.size(user_p['friends']).alias('friends'),
                       user_p['votes']['cool'].alias('vote_cool'),
                       user_p['votes']['funny'].alias('vote_funny'),
                       user_p['votes']['useful'].alias('vote_useful'),
                       )

In [86]:
df_user_p = user_p.toPandas()

In [103]:
def convert2months(yelping_since, now='2017-04'):
    [yy, ym] = yelping_since.split('-')
    [ny, nm] = now.split('-')
    return 12 * (int(ny) - int(yy)) + int(nm) - int(ym)

In [104]:
df_user_p['yelp_age'] = df_user_p.loc[:,'yelping_since'].apply(convert2months)

In [105]:
df_user_p.head()

,user_id,name,yelping_since,fans,friends,vote_cool,vote_funny,vote_useful,yelp_age
0,7cIc52lToGmKu7Hpp6rPdg,Fred,2012-04,3,65,17,8,57,60
1,tCVQyYHcmOrx2i81C47Tew,Melissa,2012-04,2,3,16,11,53,60
2,NPGgfQS-nMI1tEF1OrwzNw,Chazz,2011-05,1,7,21,80,71,71
3,FDG85sQMtHuI2a0ix9ZBcw,Mike,2011-09,10,57,171,349,414,67
4,Kf77TWS2HOJCRFl-fLiueg,Christina,2010-07,9,171,252,186,529,81


In [107]:
df = pd.merge(df_review_u, df_user_p.drop('yelping_since', axis=1), on='user_id')
df.head()

,user_id,cool,funny,useful,stars,review_count,name,fans,friends,vote_cool,vote_funny,vote_useful,yelp_age
0,--65q1FpAL_UQtVZ2PTGew,2.695364,1.907285,3.807947,4.046358,151,Nader,28,263,656,484,883,93
1,-0itF0VWVBe3k2AdfUReGA,1.734043,1.500000,2.393617,3.670213,94,Joseph,2,54,251,189,361,77
2,-2PFmYGWB6rQhZtRh6PlPA,0.183333,0.166667,0.675000,3.283333,120,M,0,10,41,91,177,85
3,-3akdU5UTDn6dwiTCO2cAw,0.736842,0.631579,1.447368,3.934211,76,April,7,133,83,62,165,82
4,-4AMRN-7RtozD9s8ZsH04g,1.644068,2.745763,2.644068,3.593220,59,Hion,33,219,1325,1674,2067,81


In [159]:
df.to_excel('yelp_user.xlsx')

## Text Processing
### Keep only df_users

In [109]:
review_p = review.filter(review.user_id.isin(df_users)) \
                 .select('user_id', 'text')

In [155]:
def FRE(list_of_records):
    
    for row in list_of_records:
        try:
            score = textstat.flesch_reading_ease(row['text'])
        except TypeError:
            continue
        
        yield (row['user_id'], score)

In [154]:
textstat.flesch_reading_ease('ss')

129.68

In [148]:
schema = StructType([StructField('user_id', StringType()),
                    StructField('FRE', FloatType())])

In [156]:
Fs = review_p.rdd.mapPartitions(FRE).collect()

In [161]:
df_FRE = sqlContext.createDataFrame(Fs, schema).toPandas()

In [162]:
df_FRE.head()

,user_id,FRE
0,JPPhyFE-UE453zA6K0TVgw,96.180000
1,fhNxoMwwTipzjO8A9LFe8Q,74.190002
2,tAKjY3bQXH51msJbOHYPmQ,83.660004
3,Sjb5e5-gKoLXueFDMc2R8Q,87.720001
4,2rSeth60_CuWN3ZJ4k41lg,84.680000


In [163]:
df_FRE.to_excel('Review_p_FRE.xlsx')

In [165]:
df_FRE_u = df_FRE.groupby('user_id').mean().reset_index()

In [167]:
df = df.merge(df_FRE_u, on='user_id')
df.head()

,user_id,cool,funny,useful,stars,review_count,name,fans,friends,vote_cool,vote_funny,vote_useful,yelp_age,FRE
0,--65q1FpAL_UQtVZ2PTGew,2.695364,1.907285,3.807947,4.046358,151,Nader,28,263,656,484,883,93,78.259934
1,-0itF0VWVBe3k2AdfUReGA,1.734043,1.500000,2.393617,3.670213,94,Joseph,2,54,251,189,361,77,69.490000
2,-2PFmYGWB6rQhZtRh6PlPA,0.183333,0.166667,0.675000,3.283333,120,M,0,10,41,91,177,85,79.785834
3,-3akdU5UTDn6dwiTCO2cAw,0.736842,0.631579,1.447368,3.934211,76,April,7,133,83,62,165,82,77.291579
4,-4AMRN-7RtozD9s8ZsH04g,1.644068,2.745763,2.644068,3.593220,59,Hion,33,219,1325,1674,2067,81,75.822373


In [168]:
df.to_excel('yelp_user.xlsx')

In [175]:
df.FRE.sort_values()[:5]

3769    17.708368
2392    22.675798
532     23.509267
393     40.380000
4320    48.444259
Name: FRE, dtype: float64

In [179]:
df.FRE.sort_values(ascending=False)[:5]

2304    100.673637
4104     99.406225
122      97.917571
4656     97.396849
637      96.144085
Name: FRE, dtype: float64

In [181]:
df[df.FRE > 96]

,user_id,cool,funny,useful,stars,review_count,name,fans,friends,vote_cool,vote_funny,vote_useful,yelp_age,FRE
122,0grN-z7ah_vqotN24HClkA,0.171429,0.128571,0.185714,3.728571,70,Joe,7,20,95,99,319,50,97.917571
637,6wpEHXevnaIZp2kUTUfhxQ,0.253521,0.070423,0.535211,3.957746,71,Jeff,0,8,23,16,62,111,96.144085
2304,Ty2QGeDJsV9zp9GmlXrCrQ,0.072727,0.254545,0.163636,3.890909,55,bobby,3,12,92,142,201,97,100.673637
4104,qsyO7LAgVVFoLhxkdBwzGg,0.415094,0.245283,0.603774,4.320755,53,Scott,3,7,31,18,54,66,99.406225
4656,ylIgegaQaJOjL4_lH2EkFA,1.534247,0.958904,1.547945,4.000000,73,Sammy,12,154,195,113,200,66,97.396849


In [177]:
df[df.FRE < 41]

,user_id,cool,funny,useful,stars,review_count,name,fans,friends,vote_cool,vote_funny,vote_useful,yelp_age,FRE
393,46xrtwjJidLUkqd-XOYg7w,0.245283,1.245283,0.433962,4.660377,53,James,4,41,26,93,36,17,40.380000
532,5lq4LkrviYgQ4LJNsBYHcA,3.759884,2.580521,5.129219,3.773385,1037,Michael,413,930,9405,6332,12901,111,23.509267
2392,VD4E91XNSP8_B5fit__u8w,0.050420,0.100840,0.176471,3.823529,119,Slade,0,0,8,17,30,19,22.675798
3769,mPXLOWxmUwmN8HGDwE-i2A,0.234694,0.102041,0.295918,3.989796,98,Carlos,1,2,31,18,43,51,17.708368


In [178]:
review_p.filter(review_p.user_id == '5lq4LkrviYgQ4LJNsBYHcA').take(5)

[Row(user_id=u'5lq4LkrviYgQ4LJNsBYHcA', text=u"Very modern, super friendly staff, classess offered to public and students alike.  Unfortunately they were between shows when we visited, but even then there was a lot to see and the instructor upstairs was quite comfortable working in front of the public.  I'd go back, especially if I lived in town."),
 Row(user_id=u'5lq4LkrviYgQ4LJNsBYHcA', text=u'For our last lunch in Pittsburgh we decided to return to the Big Burrito Group; Eleven was quite good and both Gayot and Pittsburgh Magazine compared the experience at Casbah favorably to that of Big Burrito\'s crown jewel.  Citing a mix of Mediterranean and Northern African influences with a bargain Piccola Gusto Menu for lunch and the option for indoor or al fresco dining it seemed as if Casbah was a can\'t miss option...well, sometimes things don\'t turn out quite as you expect - almost everything that transpired from the moment we arrived until the moment we left was a failure.\n\nWhen our 

In [184]:
review_p.filter(review_p.user_id == '46xrtwjJidLUkqd-XOYg7w').take(5)

[Row(user_id=u'46xrtwjJidLUkqd-XOYg7w', text=u"Definite not consistent it's either hit or miss would like to give them 5 stars but I truly believe that they didn't earn it the food can be okay great place to eat if you just want something better than fast food"),
 Row(user_id=u'46xrtwjJidLUkqd-XOYg7w', text=u'Out of all of the Lindo Michoacan and whatever locations they have Viva Zapata is the best love this place been here many many times always will have my business and the consistency from the staff to customer service is off the charts I wish I can give you 10 stars but the best I could do with this v definition this place out if you were on Pecos and Craig'),
 Row(user_id=u'46xrtwjJidLUkqd-XOYg7w', text=u"I always thought the name of this place was extremely funny I always thought shouldn't be called for her long laugh out loud five stars did a good"),
 Row(user_id=u'46xrtwjJidLUkqd-XOYg7w', text=u"I choose you this place 3 stars because the following reasons, when I ask for help 

In [185]:
review.filter(review.user_id == '46xrtwjJidLUkqd-XOYg7w').take(5)

[Row(business_id=u'3_Imz6IarFowUFjQOoibrg', date=u'2015-12-29', review_id=u'N4exWf1VhA99zgGzdV8hZQ', stars=4, text=u"Definite not consistent it's either hit or miss would like to give them 5 stars but I truly believe that they didn't earn it the food can be okay great place to eat if you just want something better than fast food", type=u'review', user_id=u'46xrtwjJidLUkqd-XOYg7w', votes=Row(cool=0, funny=0, useful=0)),
 Row(business_id=u'rG_8t33D2ADcbBqcXqwV0Q', date=u'2015-12-23', review_id=u'Utl4UtvMTJopeFRjfNMx4A', stars=5, text=u'Out of all of the Lindo Michoacan and whatever locations they have Viva Zapata is the best love this place been here many many times always will have my business and the consistency from the staff to customer service is off the charts I wish I can give you 10 stars but the best I could do with this v definition this place out if you were on Pecos and Craig', type=u'review', user_id=u'46xrtwjJidLUkqd-XOYg7w', votes=Row(cool=0, funny=2, useful=0)),
 Row(busi

In [188]:
df[df.stars > 4.8]

,user_id,cool,funny,useful,stars,review_count,name,fans,friends,vote_cool,vote_funny,vote_useful,yelp_age,FRE
740,8IXV9otfU4bKiU-dvcxW-A,0.437500,0.296875,0.593750,4.812500,64,Caroline,0,10,41,23,70,90,80.483126
1168,EP2rKr6NsXal0Yy7lzm9sw,0.428571,0.174603,0.793651,4.825397,63,Bianca,3,11,35,17,77,56,80.864286
2146,Rro65XbBIgTc392wf4oAMg,3.698113,1.905660,3.981132,4.924528,53,Lesta,13,79,269,139,297,119,73.332265
2208,S_S06oA6xGAPr-HC1kLVlQ,0.615385,0.230769,0.923077,4.871795,78,Robert,1,2,86,38,115,67,70.890770
2328,UGxtR5-KuGLMqoGI0ICxfQ,0.250000,0.083333,0.444444,4.833333,108,Carlos,2,15,34,11,68,23,79.257130
